In [8]:
%pip install einops
%pip install peft
%pip install trl
%pip install tensorboard
%pip install -U transformers
# %pip install -U accelerate datasets 
# %pip install -q https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.1-py3-none-win_amd64.whl
# %pip install tokenizers==0.15.0
# %pip install torch==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
# %pip install torchaudio==2.1.2+cu121 --index-url https://download.pytorch.org/whl/cu121
# %pip install torchvision==0.16.2+cu121 --index-url https://download.pytorch.org/whl/cu121
# %pip install transformers==4.35.2
# %pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.37.2-py3-none-any.whl.metadata (129 kB)
Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Note: you may need to restart the kernel to use updated packages.


In [168]:
import os
from dataclasses import dataclass, field
from typing import Optional
import json

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from tqdm.notebook import tqdm

from trl import SFTTrainer

In [179]:
dataset = load_dataset("json", data_files="./2011_2023_event_des.json", field='data', split='all')
dataset = dataset.train_test_split(test_size=0.1)


Found cached dataset json (C:/Users/danm/.cache/huggingface/datasets/json/default-75cc6199393bf5a9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [170]:
dataset

DatasetDict({
    train: Dataset({
        features: ['train'],
        num_rows: 717166
    })
    test: Dataset({
        features: ['train'],
        num_rows: 79686
    })
})

In [183]:
tokenizer = AutoTokenizer.from_pretrained("../models/phi-2-mlb/tokenizer_merged", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [184]:
# tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
tokenizer.pad_token = "<PAD>"
# tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

In [185]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

In [186]:
model = AutoModelForCausalLM.from_pretrained(
        "../models/phi-2-mlb", 
        quantization_config=bnb_config, 
        device_map = 'auto',
        trust_remote_code=True,
        use_auth_token=True,
    )
model.config.eos_token_id = tokenizer.eos_token_id

Could not locate the configuration_phi.py inside ../models/phi-2-mlb.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\danm\AppData\Local\Temp\ipykernel_42592\3659864661.py", line 1, in <module>
    model = AutoModelForCausalLM.from_pretrained(
  File "c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\transformers\models\auto\auto_factory.py", line 526, in from_pretrained
  File "c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\transformers\models\auto\configuration_auto.py", line 1109, in from_pretrained
  File "c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\transformers\dynamic_module_utils.py", line 488, in get_class_from_dynamic_module
    final_module = get_cached_module_file(
  File "c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-pac

Embedding(50299, 2560)

In [10]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) 

lora_config = LoraConfig(
    r=32, 
    lora_alpha=32, 
    target_modules = [ "q_proj", "k_proj", "v_proj", "dense" ],
    modules_to_save = ["lm_head", "embed_tokens"],
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

model.config.use_cache = False

In [8]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj','dense','fc1','fc2',]
)

In [11]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [18]:
from transformers import TrainingArguments, Trainer

# dataset-specific parameters
bs=8    # batch size for training
bs_eval=16    # batch size for evaluation
ga_steps=16  # gradient accumulation steps
lr=0.00002  # learning rate
epochs=1

steps_per_epoch=len(dataset["train"])//(bs*ga_steps)

args = TrainingArguments(
    output_dir="../models/phi-2-mlb",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs_eval,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch//2,    # 2 evals per epoch
    save_steps=steps_per_epoch//100,
    save_total_limit=3,     # save once per epoch
    gradient_accumulation_steps=ga_steps,
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",      # val_loss will go nan with paged_adamw_8bit
    learning_rate=lr,
    group_by_length=False,
    bf16=True,        
    ddp_find_unused_parameters=False,
    
)



In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=lora_config,
    dataset_text_field="train",
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=args,
    packing=False,
)

In [ ]:
trainer.train()

In [77]:
trainer.save_model("../models/phi-2-mlb/")

In [ ]:
trainer.train(resume_from_checkpoint="../models/phi-2-mlb/checkpoint-1344")

In [148]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

# base model
base_path="../models/phi-2"  

# adapters: path to folder with adapter_model.safetensors
adapter_path="../models/phi-2-mlb/checkpoint-1344" 

# # where to save merged model
save_to="../models/phi-2-mlb/"       

# # Load model and tokenizer
# base_model = AutoModelForCausalLM.from_pretrained(
#     base_path,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )

# tokenizer = AutoTokenizer.from_pretrained(base_path)

# # Add/set tokens same tokens to base model before merging, like we did before training  
# tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
# tokenizer.pad_token = "<PAD>"
# tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

# base_model.config.eos_token_id = tokenizer.eos_token_id

# # Set a default Generation configuration: Llama precise
# generation_config = GenerationConfig(
#     max_new_tokens=100, 
#     temperature=0.7,
#     top_p=0.1,
#     top_k=40,
#     repetition_penalty=1.18,
#     do_sample=True,
#     pad_token_id=tokenizer.pad_token_id,
#     eos_token_id=tokenizer.eos_token_id,
# )

# Load LoRA and merge
merged_model = PeftModel.from_pretrained(model, adapter_path)
merged_model = merged_model.merge_and_unload()

# merged_model.save_pretrained(save_to, safe_serialization=True, max_shard_size='4GB')
# tokenizer.save_pretrained(save_to)
# generation_config.save_pretrained(save_to)

c:\Users\danm\Downloads\oobabooga_windows\installer_files\env\lib\site-packages\peft\tuners\lora\bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

model_path="../models/phi-2-mlb/"   

prompt="Instruct: {\"input\": {\"pitcher\": {\"id\": 460024, \"name\": \"luke hochevar\"}, \"batter\": {\"id\": 110029, \"name\": \"bobby abreu\"}, \"p_throws\": \"R\", \"stand\": \"L\", \"inning_topbot\": \"Top\", \"inning\": 1, \"outs_when_up\": 1, \"on_1b\": \"\", \"on_2b\": {\"id\": 435062, \"name\": \"howie kendrick\"}, \"on_3b\": \"\", \"home_score\": 0, \"away_score\": 0}}? \n"

# model = AutoModelForCausalLM.from_pretrained(
#         model_path,    
#         torch_dtype=torch.bfloat16,
#         device_map="auto"
#     )
# tokenizer = AutoTokenizer.from_pretrained(model_path) 

input_tokens = tokenizer(prompt, return_tensors="pt").to("cuda")
output_tokens = merged_model.generate(**input_tokens, max_new_tokens=512)

output = tokenizer.decode(
    output_tokens[0][len(input_tokens[0]):],
    skip_special_tokens=True
    )               

print(output)